In [2]:
from skimage.restoration import denoise_nl_means
import imageio
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
import numpy as np
import visvis as vv
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu, rank, threshold_adaptive, wiener, threshold_isodata
from skimage.util import img_as_ubyte

import matplotlib.pyplot as plt


filename = 'route27north1.mp4'
reader = imageio.get_reader(filename)



In [3]:
for i, im in enumerate(reader):
    #print('Mean of frame %i is %1.1f' % (i, im.mean()))
    if i>430:
        test_image = img_as_ubyte(rgb2gray(im))
        break
#vv.imshow(test_image)

print(test_image.shape)

#test_image2 = denoise_nl_means(test_image,patch_size=11)
#print(test_image2.shape)

(480L, 640L)


C:\Users\Isabel\Anaconda2\lib\site-packages\skimage\util\dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint8
  "%s to %s" % (dtypeobj_in, dtypeobj))


In [4]:
thresh=threshold_otsu(test_image)
otsu_image = test_image > thresh*3

print(thresh)
thresh2 = threshold_isodata(test_image,6)
print(thresh2)
bin_image1 = test_image > thresh2*4

#bin_image1 = threshold_adaptive(test_image, 31, 'mean')
#func = lambda arr: arr.mean()
#bin_image2 = threshold_adaptive(test_image, 31, 'gaussian')
print(test_image.mean())

fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
plt.tight_layout()
bins_list = [otsu_image, bin_image1, test_image]
titles = ['Otsu threshold', 'ISODATA threshold',
          'Original image']

sequence = zip(bins_list, titles)
axes = axes.ravel()
for bin_img, title in sequence:
    ax = axes[0]
    axes = axes[1:]
    ax.set_title(title)
    ax.imshow(bin_img, cmap='Greys_r')
    ax.set_axis_off()

plt.show()

next_image = bin_image1

54
53
30.4766373698


In [5]:

blobs_log = blob_log(next_image, max_sigma=30, num_sigma=10, threshold=.1)

# Compute radii in the 3rd column.
if blobs_log.shape>0:
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    print(blobs_log[:, 2].mean())
    blobs_log = blobs_log[blobs_log[:,2]>2]

blobs_dog = blob_dog(next_image, max_sigma=30, threshold=.1)
if blobs_dog.shape>0:
    blobs_dog[:, 2] = blobs_dog[:, 2] * sqrt(2)
    print(blobs_dog[:, 2].mean())
    blobs_dog = blobs_dog[blobs_dog[:,2]>2]

blobs_doh = blob_doh(next_image, max_sigma=30, threshold=.01)
print(blobs_doh[:, 2].mean())
blobs_doh = blobs_doh[blobs_doh[:,2]>2]

blobs_list = [blobs_log, blobs_dog, blobs_doh]
colors = ['yellow', 'lime', 'red']
titles = ['Laplacian of Gaussian', 'Difference of Gaussian',
          'Determinant of Hessian']
sequence = zip(blobs_list, colors, titles)

fig, axes = plt.subplots(1, 3, figsize=(14, 4), sharex=True, sharey=True,
                         subplot_kw={'adjustable': 'box-forced'})
plt.tight_layout()

axes = axes.ravel()
for blobs, color, title in sequence:
    ax = axes[0]
    axes = axes[1:]
    ax.set_title(title)
    ax.imshow(test_image, cmap='Greys_r')
    ax.set_axis_off()
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax.add_patch(c)

plt.show()

2.37356311346
6.73018923545
3.51490514905
